# 국가별 알콜 섭취량 데이터 분석
- 데이터 읽기
- 데이터 정보 확인
- 컬럼명 한글로 변경
    - 원본 데이터 컬럼 정보
    - country : 국가명
    - beer_servings : 맥주 소비량
    - spirit_servings : 증류주 소비량(독주, 알코올 함량 20% 이상의 설탕을 첨가하지 않은 증류주)
    - wine_servings : 와인 소비량
    - total_litres_of_pure_alcohol : 순수 알코올
    - continent : 대륙
- 결측치 확인
    - `대륙`에 대한 정보가 부족한 것을 확인할 수 있다.
    - drinks.csv 파일을 열어 원인을 확인한다.
    - 대륙의 종류
        - 아시아, 유럽, 아프리카, 북아메리카, 남아메리카, 오세아니아
        - AS, EU, AF, NA, SA, OC
- 결측치 해결
    - 'NA'를 NaN(결측치)로 읽는 것을 방지
    - pd.read_csv 함수의 na_filter=False 파라메터 : 결측치를 읽어올지 여부 지정
    - 옵션 확인 : https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html 
- 통계값 구하기
    - 전세계 맥주, 증류주, 와인, 알코올의 소비에 대한 '평균', '최대값', '최소값' 구하기
- 주류 소비량 비교
    - 와인 소비량이 맥주 + 증류주 소비량보다 큰 나라를 검색해,'대륙'을 기준으로 정렬
    - 맥주 소비량이 230 초과이면서, 와인 소비량이 230 초과인 나라를 검색
    - 전체 평균보다 많은 알코올을 섭취하는 대륙과 알코올 평균 소비량 검색
    - 맥주를 가장 많이 소비하는 국가 확인
    - 증류주를 가장 많이 소비하는 국가 확인
    - 평균 맥주 소비량이 가장 많은 대륙 확인
- 데이터 프레임 조작
    - 국가별 주류 소비량 합계를 구해 새로운 컬럼 ('주류소비량') 추가
    - 주류 소비량 대비 알코올 소비량 비율을 구해 새로운 컬럼 ('알코올비율') 추가
        - 알코올비율 = '알코올' / '주류소비량'
        - 알코올비율'이 NaN인 것에 대해 0으로 채우기
    - 알코올비율이 높은 국가(술을 독하게 마시는 국가) TOP5의 ['국가', '주류소비량', '알코올비율'] 정보 조회
    - DataFrame의 index를 ['대륙', '국가'] 로 지정하고, 대륙별, 국가명으로  정렬하여 df로 저장
    - 전세계 평균 소비량과 한국의 소비량을 비교하는 DataFrame 만들기
        <pre>
               맥주	    증류주	     와인	      알코올	  주류소비량	알코올비율
     국가						
South Korea	140.000000	16.000000	9.000000	9.800000	165.000000	0.059394
World Wide	106.160622	80.994819	49.450777	4.717098	236.606218	0.026115
</pre>
    - pivot_table을 사용하여 대륙별(index), '맥주'와 '와인'의 mean, median, max 값을 구한다.

## 데이터 읽기
csv 파일 가져오기 
- csv 파일은 컬럼 구분이 콤마(,) 로 이루어진 파일
- df = pd.read_csv(파일이름) : csv 형식의 파일을 읽어 DataFrame 객체로 가져옴
- DataFrame.to_csv(파일이름, index=True) : DataFrame을 csv 파일로 저장, index는 제외하고 저장하기 위해서는 index=False로 지정

In [136]:
import pandas as pd
import numpy as np

df = pd.read_csv('drinks.csv',na_filter=False)
df.head()

,country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol,continent
0,Afghanistan,0,0,0,0.0,AS
1,Albania,89,132,54,4.9,EU
2,Algeria,25,0,14,0.7,AF
3,Andorra,245,138,312,12.4,EU
4,Angola,217,57,45,5.9,AF


## 컬럼명 한글로 변경

In [138]:
df.rename(columns = {'country':'국가명',
              'beer_servings':'맥주',
              'spirit_servings':'증류주',
              'wine_servings':'와인',
              'total_litres_of_pure_alcohol':'순수알콜',
              'continent': '대륙'}, inplace=True)
df.head(5)



,국가명,맥주,증류주,와인,순수알콜,대륙
0,Afghanistan,0,0,0,0.0,AS
1,Albania,89,132,54,4.9,EU
2,Algeria,25,0,14,0.7,AF
3,Andorra,245,138,312,12.4,EU
4,Angola,217,57,45,5.9,AF


In [137]:
df.columns = ['국가','맥주''증류주','와인','순수알콜','대륙']

ValueError: Length mismatch: Expected axis has 6 elements, new values have 5 elements

## 결측치 확인

In [141]:
checked_df = df.isna().sum()
checked_df

df['대륙'].unique()

array(['AS', 'EU', 'AF', 'NA', 'SA', 'OC'], dtype=object)

## 결측치 해결

- 'NA'를 NaN(결측치)로 읽는 것을 방지
    - pd.read_csv(파일이름, na_filter=False) : 결측치를 읽어올지 여부 지정
    - 옵션 확인 : https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html 


- 결측치를 다른 값으로 채우기 함
    - DataFrame.fillna(값) 또는 Series.fillna(값) : 결측값을 특정 값으로 채우기 
    - 연속형 데이터 : 0으로 채우기, 평균값으로 채우기, 범주별 평균값 채우기 등
    - 범주형 데이터 : 다른 범주로 만들어 채우기 

In [ ]:
df = df.fillna('NA')
df.isna().sum()
df['대륙'].unique()

## 통계값 구하기
- DataFrame.describe() : 컬럼별 count, mean, std, min, max, Q1, Q2, Q3 등의 통계값 구하기

In [10]:
df.describe()

,맥주,증류주,와인,순수알콜
count,193.000000,193.000000,193.000000,193.000000
mean,106.160622,80.994819,49.450777,4.717098
std,101.143103,88.284312,79.697598,3.773298
min,0.000000,0.000000,0.000000,0.000000
25%,20.000000,4.000000,1.000000,1.300000
50%,76.000000,56.000000,8.000000,4.200000
75%,188.000000,128.000000,59.000000,7.200000
max,376.000000,438.000000,370.000000,14.400000


In [142]:
# describe(): 기본적으로 수치형 데잍터에 대한 통계를 보여준다.
# 데이터 타입에 상관 없이 모든 컬럼에 대한 통계값 확인:
# include='all' 속성을 지정 해준다.
df.describe(include='all')

,국가명,맥주,증류주,와인,순수알콜,대륙
count,193,193.000000,193.000000,193.000000,193.000000,193
unique,193,NaN,NaN,NaN,NaN,6
top,Afghanistan,NaN,NaN,NaN,NaN,AF
freq,1,NaN,NaN,NaN,NaN,53
mean,NaN,106.160622,80.994819,49.450777,4.717098,NaN
std,NaN,101.143103,88.284312,79.697598,3.773298,NaN
min,NaN,0.000000,0.000000,0.000000,0.000000,NaN
25%,NaN,20.000000,4.000000,1.000000,1.300000,NaN
50%,NaN,76.000000,56.000000,8.000000,4.200000,NaN
75%,NaN,188.000000,128.000000,59.000000,7.200000,NaN


## 주류 소비량 비교

In [161]:
# 와인 소비량이 맥주 + 증류주 소비량보다 큰 나라를 검색해,'대륙'을 기준으로 정렬
df[df['와인'] > (df['맥주']+df['증류주'])].sort_values('대륙',ascending=True).head(3)

,국가명,맥주,증류주,와인,순수알콜,대륙,주류소비량
55,Equatorial Guinea,92,0,233,5.8,AF,325
148,Sao Tome & Principe,56,38,140,4.2,AF,234
92,Laos,62,0,123,6.2,AS,185


In [146]:
# 맥주 소비량이 230 초과이면서, 와인 소비량이 230 초과인 나라를 검색
df[np.logical_and(df['맥주'] > 230, df['와인'] > 230)]

,국가명,맥주,증류주,와인,순수알콜,대륙
3,Andorra,245,138,312,12.4,EU
99,Luxembourg,236,133,271,11.4,EU
156,Slovenia,270,51,276,10.6,EU


In [154]:
# 전체 평균보다 많은 알코올을 섭취하는 대륙과 알코올 평균 소비량 검색
total_mean = df['순수알콜'].mean()
print(f'전체 알콜 평균 소비량:{total_mean:.2f}')

grouped = df.groupby('대륙')['순수알콜'].mean().sort_values(ascending=False)
grouped[grouped > total_mean]

전체 알콜 평균 소비량:4.72


대륙
EU    8.617778
SA    6.308333
NA    5.995652
Name: 순수알콜, dtype: float64

In [158]:
# 맥주를 가장 많이 소비하는 국가 확인
df[df['맥주'].max() == df['맥주']]
df[df['맥주'].max() == df['맥주']][['국가명','맥주']]
# 증류주를 가장 많이 소비하는 국가 확인
df[df['증류주'].max() == df['증류주']]


,국가명,맥주
117,Namibia,376


In [159]:
# 평균 맥주 소비량이 가장 많은 대륙 확인
grouped = df.groupby('대륙')['맥주'].mean()
grouped.sort_values(ascending=False).head(1)

대륙
EU    193.777778
Name: 맥주, dtype: float64

## 데이터 프레임 조작

In [164]:
# 국가별 주류 소비량 합계를 구해 새로운 컬럼 ('주류소비량') 추가
df['주류소비량'] = df[['맥주','증류주','와인']].sum(axis=1)
# df['주류소비량'] = df.loc[:,'맥주':'와인'].sum(axis=1)
df.head(3)

,국가명,맥주,증류주,와인,순수알콜,대륙,주류소비량,알코올비율
0,Afghanistan,0,0,0,0.0,AS,0,NaN
1,Albania,89,132,54,4.9,EU,275,0.02
2,Algeria,25,0,14,0.7,AF,39,0.02


In [165]:
# 주류 소비량 대비 알코올 소비량 비율을 구해 새로운 컬럼 ('알코올비율') 추가
# 알코올비율 = '알코올' / '주류소비량'
df['알코올비율'] = round(df['순수알콜'] / df['주류소비량'], 2)
df.head(3)

,국가명,맥주,증류주,와인,순수알콜,대륙,주류소비량,알코올비율
0,Afghanistan,0,0,0,0.0,AS,0,NaN
1,Albania,89,132,54,4.9,EU,275,0.02
2,Algeria,25,0,14,0.7,AF,39,0.02


In [167]:
# 알코올비율'이 NaN인 것에 대해 0으로 채우기
df.isna().sum()
df['알코올비율'] =df['알코올비율'].fillna(0)
df.head(3)

,국가명,맥주,증류주,와인,순수알콜,대륙,주류소비량,알코올비율
0,Afghanistan,0,0,0,0.0,AS,0,0.00
1,Albania,89,132,54,4.9,EU,275,0.02
2,Algeria,25,0,14,0.7,AF,39,0.02


In [173]:
# 알코올비율이 높은 국가(술을 독하게 마시는 국가) 
temp = df[['국가명','주류소비량','알코올비율']].sort_values('알코올비율',ascending=False).head(3)
temp.set_index('국가명' , inplace=True)
temp

,주류소비량,알코올비율
국가명,,
Gambia,9,0.27
Sierra Leone,30,0.22
Nigeria,49,0.19


In [170]:
# TOP5의 ['국가', '주류소비량', '알코올비율'] 정보 조회
df[df['알코올비율'].max() == df['알코올비율']]

,국가명,맥주,증류주,와인,순수알콜,대륙,주류소비량,알코올비율
63,Gambia,8,0,1,2.4,AF,9,0.27


In [174]:
# DataFrame의 index를 ['대륙', '국가명'] 로 지정하고, 대륙별, 국가명으로  정렬하여 df로 저장
df2 = df.set_index(['대륙','국가명'])
df2.head(3)

,,맥주,증류주,와인,순수알콜,주류소비량,알코올비율
대륙,국가명,,,,,,
AS,Afghanistan,0,0,0,0.0,0,0.00
EU,Albania,89,132,54,4.9,275,0.02
AF,Algeria,25,0,14,0.7,39,0.02


In [175]:
# 전세계 평균 소비량과 한국의 소비량을 비교하는 DataFrame 만들기
korea = df[df['국가명'] == 'South Korea']
korea

,국가명,맥주,증류주,와인,순수알콜,대륙,주류소비량,알코올비율
138,South Korea,140,16,9,9.8,AS,165,0.06


In [181]:

world_wide = pd.DataFrame(df[['맥주','증류주','와인','순수알콜','주류소비량','알코올비율']].mean())
world_wide = world_wide.T
world_wide.insert(0,'국가명', 'World Wide')
world_wide


,국가명,맥주,증류주,와인,순수알콜,주류소비량,알코올비율
0,World Wide,106.160622,80.994819,49.450777,4.717098,236.606218,0.027617


In [185]:
# concat 기본은 행단위로 연결됨
compare = pd.concat([korea, world_wide],axis=0)
compare.set_index('국가명', inplace=True)
compare

,맥주,증류주,와인,순수알콜,대륙,주류소비량,알코올비율
국가명,,,,,,,
South Korea,140.000000,16.000000,9.000000,9.800000,AS,165.000000,0.060000
World Wide,106.160622,80.994819,49.450777,4.717098,NaN,236.606218,0.027617


In [190]:
# pivot_table을 사용하여 대륙별(index),
# '맥주'와 '와인'의 mean, median, max 값을 구한다.
df.pivot_table(index='대륙', values=['맥주','와인'], aggfunc=['mean','median','max'])

mean             median         max     
            맥주          와인     맥주     와인   맥주   와인
대륙                                                
AF   61.471698   16.264151   32.0    2.0  376  233
AS   37.045455    9.068182   17.5    1.0  247  123
EU  193.777778  142.222222  219.0  128.0  361  370
NA  145.434783   24.521739  143.0   11.0  285  100
OC   89.687500   35.625000   52.5    8.5  306  212
SA  175.083333   62.416667  162.5   12.0  333  221